### Run MAF on Multiple FBS_1.4 OpSims

The previous example showed how to run a metric analysis on one or two opSims.  But what you really want to do is to be able to run through *all* of them.  This notebook shows how to do that.  The next notebook will demonstrate how to analyze the results.

In [24]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

Import the sims_maf modules needed.

In [25]:
# import lsst.sim.maf moduels modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [26]:
# import convenience functions
from opsimUtils import *

#### 1. Build connections to the OpSims databases
The first step is to initiate opsim database objects and result database objects for the opsim databases that you want to run metrics on. Two paths are needed here:
1. `dbDir`: The path to the OpSim database directory
2. `outDir`: The path to the directory where you want to save the metric metadata.

By providing these two paths, the function `connect_dbs()` will initiate connections and store the metadata to the designated directory.
#### **Attention:**
You need to update the `your_username` variable with **Your Username**.

In [ ]:
your_username = '' #add your username between the quotes here

if your_username == '': #do NOT put your username here
    raise Exception('Please provide your username!')

dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.4/'
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput'.format(your_username)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [28]:
# two dictionary are returned by the following function, 
# One (opSimDbs) is a dictionary storing all database objects
# Another (resultDbs) is a dictionary consist of the objects directing MAF where to save metric metadata
# Both dictionaries are indexed by OpSim run names
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

# Collect Run names to a list
dbRuns = list(opSimDbs.keys())

You can use `help` to get more information about the provided convenience functions, e.g., `connect_dbs()`

In [29]:
help(connect_dbs)

Help on function connect_dbs in module opsimUtils:

connect_dbs(dbDir, outDir)
    Initiate database objects to all opSim databases in the provided directory.
    Returns a dictionary consisting all database connections and a dictionary holding
    the resultsDb objects.
    
    Args:
        dbDir(str): The path to the dabase directory.
        outDir(str): The path to the result database directory.
    
    Returns:
        opSimDbs(dict): A dictionary containing the OpsimDatabase objects for opsim databases in the provided directory, keys are the run names.
        resultDbs(str): A dictionary containing the ResultsDb objects for opsim databases in the provided directory, keys are the run names.



You can also check what OpSims are available in the directory

In [30]:
show_opsims(dbDir)

['alt_roll_mod2_dust_sdf_0.20_v1.4_10yrs.db',
 'baseline_v1.4_10yrs.db',
 'dcr_nham3_v1.4_10yrs.db',
 'rolling_mod3_sdf_0.20_v1.4_10yrs.db']

#### 2. Declare some metrics to run on above OpSims

In [31]:
# Airmass Metric
maxMetric = metrics.MaxMetric('airmass')
airmassSlicer = slicers.HealpixSlicer(nside=64)
airmassConstraint = 'filter = "g"'
airmassConstraint += ' and note not like "DD%"'

maxairmassSky = metricBundles.MetricBundle(maxMetric, airmassSlicer, \
                airmassConstraint)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [32]:
# Coadd Metric
coaddMetric = metrics.Coaddm5Metric()
coaddSlicer = slicers.HealpixSlicer(nside=64)
coaddConstraint = 'filter = "y"'
coaddConstraint += ' and note not like "DD%"'

CoaddY = metricBundles.MetricBundle(coaddMetric, coaddSlicer, \
                coaddConstraint)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [33]:
# set summary statistics
summaryMetrics = [metrics.MinMetric(), metrics.MedianMetric(),
                  metrics.MaxMetric(), metrics.RmsMetric()]
maxairmassSky.setSummaryMetrics(summaryMetrics)
CoaddY.setSummaryMetrics(summaryMetrics)

In [34]:
# create a bundleDict for all metrics to run on each opSim
bundleDict = {'maxairmassSky': maxairmassSky, 'CoaddY': CoaddY}

#### 3. Loop over all OpSims in dbDir and run MAF
While constructing a metricBundleGroup from a dictionary (the cell below), you will need to provide the path to a directory (`metricDataPath` in the cell below) where you would like to store the metric data (this is **DIFFERENT** than path to the metric data, `outDir`). To construct metricbundles for plotting and further analysis, this path will be needed.

In [35]:
metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/MetricData/'.format(your_username)
for run in dbRuns:
    # must set run name for each opSim to store metric data into
    # separate files
    maxairmassSky.setRunName(run)
    CoaddY.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()

Querying database SummaryAllProps with constraint filter = "y" and note not like "DD%" for columns ['fieldDec', 'fieldRA', 'fiveSigmaDepth']
Found 390345 visits
Running:  ['CoaddY']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and note not like "DD%" for columns ['fieldDec', 'fieldRA', 'airmass']
Found 195088 visits
Running:  ['maxairmassSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "y" and note not like "DD%" for columns ['fieldDec', 'fieldRA', 'fiveSigmaDepth']
Found 384344 visits
Running:  ['CoaddY']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and note not like "DD%" for columns ['fieldDec', 'fieldRA', 'airmass']
Found 189244 visits
Running:  ['maxairmassSk

Now you have computed your metric for all the opSims.  To look at those results, go through [View_Results.ipynb](./View_Results.ipynb).